In [1]:
import cv2 as cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt

In [2]:
mp_holistic = mp.solutions.holistic
mp_pose=mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)           # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)               # Draw pose connections
    #mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)          # Draw left hand connections
    #mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
   
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    

In [6]:
def get_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)

In [7]:
out = np.array(['good left half','good right half','bad left half', 'bad right half'])
sq_len=20
#fps_a= cap.get(cv2.CAP_PROP_FPS)
#frame_count=cap.get(cv2.CAP_PROP_FRAME_COUNT)
#no_sq=round(frame_count/fps_a)
no_sq=4

In [8]:
vid_q=['lr.mp4','rl1.mp4','rl2.mp4']

# LEFT RIGHT GOOD DATA:

In [9]:
cap = cv2.VideoCapture('Trainingvideosfinal/good/vid1/lr.mp4')
# used to record the time when we processed last frame
prev_frame_time = 0

# used to record the time at which we processed current frame
new_frame_time = 0
# VIDEO FEED

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

    differenceHipsList = []
    differenceShouldersList = []
    leftshoulderList = []
    rightshoulderList = []
    lsp=[]
    rsp=[]
    lhp=[]
    rhp=[]
    lefthipList = []
    righthipList = []
    midpointsholder_l=[]
    midpointhip_l=[]
    spinelength_l=[]
    s_len=[]
    h_len=[]
    ratio_l=[]
    framecounter = 0
    while cap.isOpened():

        ret, frame = cap.read()
        if not ret:
            break
        if frame is not None:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


            font = cv2.FONT_HERSHEY_SIMPLEX
            new_frame_time = time.time()
        # Calculating the fps

        # fps will be number of frame processed in given time frame
        # since their will be most of time error of 0.001 second
        # we will be subtracting it to get more accurate result
            fps = 1 / (new_frame_time - prev_frame_time)
            prev_frame_time = new_frame_time

        # converting the fps into integer
            fps = int(fps)

        # converting the fps to string so that we can display it on frame
        # by using putText function
            fps_1 = str(fps)

        # putting the FPS count on the frame
            cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



            try:
                landmarks = results.pose_landmarks.landmark
                leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                differenceHips = np.absolute(righthip.z - lefthip.z)
                differenceHipsList.append(differenceHips)
                differenceShouldersList.append(differenceShoulders)
                leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                lefthipList.append([lefthip.x, lefthip.y])
                righthipList.append([righthip.x, righthip.y])

            except:

                pass




            differenceHipsAverage = np.mean(differenceHipsList)
            differenceShouldersAverage = np.mean(differenceShouldersList)


            slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
            slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

            rate_slope_shoulder = slope_shoulder * fps

            rate_slope_shoulder_length = differenceShouldersAverage * fps
            midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
            midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
            leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
            rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
            lefthip_poin=np.array([lefthip.x,lefthip.y])
            righthip_poin=np.array([righthip.x,righthip.y])
            midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
            midpointhip_p_x = (lefthip.x+righthip.x)/2
            midpointhip_p_y = (lefthip.y+righthip.y)/2
            midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])

            s_len_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)
            h_len_v=np.linalg.norm(lefthip_poin-righthip_poin)
            ratio_v=s_len_v/h_len_v
            spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
            framecounter += 1



        #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
        #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
        #From there, we should be able to detect in which phase the subject is in each frame.

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                    )


            cv2.imshow('Mediapipe Feed', image)
            #image_l.append(image)
            #print(image.shape)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length),s_len.append(s_len_v),h_len.append(h_len_v)

cap.release()
#print(midpointhip_l)
cv2.destroyAllWindows()

In [11]:
data={'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l
      
        }
dataF12=pd.DataFrame(data)

# RIGHT LEFT GOOD DATA:

In [13]:
cap = cv2.VideoCapture('Trainingvideosfinal/good/vid1/rl1.mp4')
# used to record the time when we processed last frame
prev_frame_time = 0

# used to record the time at which we processed current frame
new_frame_time = 0
# VIDEO FEED

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    slope_shoulder_l,slope_hip_l,rate_slope_shoulder_l,rate_slope_shoulder_length_l=[],[],[],[]

    differenceHipsList = []
    differenceShouldersList = []
    leftshoulderList = []
    rightshoulderList = []
    lsp=[]
    rsp=[]
    lhp=[]
    rhp=[]
    lefthipList = []
    righthipList = []
    midpointsholder_l=[]
    midpointhip_l=[]
    spinelength_l=[]
    s_len=[]
    h_len=[]
    ratio_l=[]
    framecounter = 0
    while cap.isOpened():

        ret, frame = cap.read()
        if not ret:
            break
        if frame is not None:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


            font = cv2.FONT_HERSHEY_SIMPLEX
            new_frame_time = time.time()
        # Calculating the fps

        # fps will be number of frame processed in given time frame
        # since their will be most of time error of 0.001 second
        # we will be subtracting it to get more accurate result
            fps = 1 / (new_frame_time - prev_frame_time)
            prev_frame_time = new_frame_time

        # converting the fps into integer
            fps = int(fps)

        # converting the fps to string so that we can display it on frame
        # by using putText function
            fps_1 = str(fps)

        # putting the FPS count on the frame
            cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



            try:
                landmarks = results.pose_landmarks.landmark
                leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                differenceShoulders = np.linalg.norm(rightshoulder.z - leftshoulder.z)
                righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                differenceHips = np.linalg.norm(righthip.z - lefthip.z)
                differenceHipsList.append(differenceHips)
                differenceShouldersList.append(differenceShoulders)
                leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                lefthipList.append([lefthip.x, lefthip.y])
                righthipList.append([righthip.x, righthip.y])

            except:

                pass




            differenceHipsAverage = np.mean(differenceHipsList)
            differenceShouldersAverage = np.mean(differenceShouldersList)


            slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
            slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

            rate_slope_shoulder = slope_shoulder * fps

            rate_slope_shoulder_length = differenceShouldersAverage * fps
            midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
            midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
            leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
            rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
            lefthip_poin=np.array([lefthip.x,lefthip.y])
            righthip_poin=np.array([righthip.x,righthip.y])
            midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
            midpointhip_p_x = (lefthip.x+righthip.x)/2
            midpointhip_p_y = (lefthip.y+righthip.y)/2
            midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])

            s_len_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)
            h_len_v=np.linalg.norm(lefthip_poin-righthip_poin)
            ratio_v=s_len_v/h_len_v
            spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
            framecounter += 1



        #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
        #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
        #From there, we should be able to detect in which phase the subject is in each frame.

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                    )


            cv2.imshow('Mediapipe Feed', image)
            #image_l.append(image)
            #print(image.shape)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y]),lhp.append([lefthip.x,lefthip.y]),rhp.append([righthip.x,righthip.y]),ratio_l.append(ratio_v),spinelength_l.append(spinelength_v),midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y]),midpointhip_l.append([midpointhip_p_x,midpointhip_p_y]),slope_shoulder_l.append(slope_shoulder),slope_hip_l.append(slope_hip),rate_slope_shoulder_l.append(rate_slope_shoulder),rate_slope_shoulder_length_l.append(rate_slope_shoulder_length),s_len.append(s_len_v),h_len.append(h_len_v)

cap.release()
#print(midpointhip_l)
cv2.destroyAllWindows()

In [15]:
cap = cv2.VideoCapture('Trainingvideosfinal/good/vid1/rl2.mp4')
# used to record the time when we processed last frame
prev_frame_time = 0

# used to record the time at which we processed current frame
new_frame_time = 0
# VIDEO FEED

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    framecounter = 0
    while cap.isOpened():

        ret, frame = cap.read()
        if not ret:
            break
        if frame is not None:
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)


            font = cv2.FONT_HERSHEY_SIMPLEX
            new_frame_time = time.time()
        # Calculating the fps

        # fps will be number of frame processed in given time frame
        # since their will be most of time error of 0.001 second
        # we will be subtracting it to get more accurate result
            fps = 1 / (new_frame_time - prev_frame_time)
            prev_frame_time = new_frame_time

        # converting the fps into integer
            fps = int(fps)

        # converting the fps to string so that we can display it on frame
        # by using putText function
            fps_1 = str(fps)

        # putting the FPS count on the frame
            cv2.putText(image, fps_1, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)



            try:
                landmarks = results.pose_landmarks.landmark
                leftshoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
                rightshoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                differenceShoulders = np.absolute(rightshoulder.z - leftshoulder.z)
                righthip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
                lefthip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
                differenceHips = np.absolute(righthip.z - lefthip.z)
                differenceHipsList.append(differenceHips)
                differenceShouldersList.append(differenceShoulders)
                leftshoulderList.append([leftshoulder.x, leftshoulder.y])
                rightshoulderList.append([rightshoulder.x, rightshoulder.y])
                lefthipList.append([lefthip.x, lefthip.y])
                righthipList.append([righthip.x, righthip.y])

            except:

                pass




            differenceHipsAverage = np.mean(differenceHipsList)
            differenceShouldersAverage = np.mean(differenceShouldersList)


            slope_shoulder = np.arctan((leftshoulderList[framecounter][1] - rightshoulderList[framecounter][1]) / (leftshoulderList[framecounter][0] - rightshoulderList[framecounter][0]))
            slope_hip = np.arctan((lefthipList[framecounter][1] - righthipList[framecounter][1]) / (lefthipList[framecounter][0] - righthipList[framecounter][0]))

            rate_slope_shoulder = slope_shoulder * fps

            rate_slope_shoulder_length = differenceShouldersAverage * fps
            midpointsholder_p_x = (leftshoulder.x+rightshoulder.x)/2
            midpointsholder_p_y = (leftshoulder.y+rightshoulder.y)/2
            leftsholder_poin=np.array([leftshoulder.x,leftshoulder.y])
            rightsholder_poin=np.array([rightshoulder.x,rightshoulder.y])
            lefthip_poin=np.array([lefthip.x,lefthip.y])
            righthip_poin=np.array([righthip.x,righthip.y])
            midpointsholder_poin=np.array([midpointsholder_p_x,midpointsholder_p_y])
            midpointhip_p_x = (lefthip.x+righthip.x)/2
            midpointhip_p_y = (lefthip.y+righthip.y)/2
            midpointhip_poin=np.array([midpointhip_p_x,midpointhip_p_y])

            s_len_v=np.linalg.norm(leftsholder_poin-rightsholder_poin)
            h_len_v=np.linalg.norm(lefthip_poin-righthip_poin)
            ratio_v=s_len_v/h_len_v
            spinelength_v=np.linalg.norm(midpointsholder_poin-midpointhip_poin)
            framecounter += 1



        #To solve shoulder fluctuation problem with being stuck on one side, we should be able to detect in which phase of the gait cycle the subject is in each frame.
        #In order to determine phase of gait cycle, we have to know the neutral position (standing neutral position) and the corresponding bones positions.
        #From there, we should be able to detect in which phase the subject is in each frame.

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                    )


            cv2.imshow('Mediapipe Feed', image)
            #image_l.append(image)
            #print(image.shape)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
            lsp.append([leftshoulder.x,leftshoulder.y]),rsp.append([rightshoulder.x,rightshoulder.y])
            lhp.append([lefthip.x,lefthip.y])
            rhp.append([righthip.x,righthip.y])
            ratio_l.append(ratio_v)
            spinelength_l.append(spinelength_v)
            midpointsholder_l.append([midpointsholder_p_x,midpointsholder_p_y])
            midpointhip_l.append([midpointhip_p_x,midpointhip_p_y])
            slope_shoulder_l.append(slope_shoulder)
            slope_hip_l.append(slope_hip)
            rate_slope_shoulder_l.append(rate_slope_shoulder)
            rate_slope_shoulder_length_l.append(rate_slope_shoulder_length)
            s_len.append(s_len_v),h_len.append(h_len_v)

cap.release()
#print(midpointhip_l)
cv2.destroyAllWindows()

In [17]:

data={'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l
        }
dataF22=pd.DataFrame(data)

# LEFT RIGHT BAD DATA:

In [ ]:
data={'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l
      
        }
dataF32=pd.DataFrame(data)

# RIGHT LEFT BAD DATA

In [ ]:
data={'difference_shoulder': differenceShouldersList,
      'difference_hip': differenceHipsList,
      'Spine_Length':spinelength_l,
      'Shoulder_Length':s_len,
      'Slope_shoulder':slope_shoulder_l,
      'Slope_hip':slope_hip_l
      
        }
dataF42=pd.DataFrame(data)

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(out)}

In [ ]:
a=label_map.items()

In [ ]:
dataFR = [np.array(dataF12),np.array(dataF22),np.array(dataF32),np.array(dataF42)]

In [ ]:
seq, labels =[], []
for i in range(len(out)):
    seq.append(dataFR[i])
    seq[i]=seq[i][:30]
    labels.append(label_map[out[i]])

In [ ]:
seq=np.array(seq)

In [ ]:
x = np.array(seq)
y = to_categorical(labels).astype(int)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, shuffle=True, random_state=27)

In [ ]:
type(x_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
#log_dir = os.path.join('Logs')
#tb_callback = EarlyStopping(monitor='val_loss', patience=30, mode='min', restore_best_weights=True)
#e_callback = TensorBoard(log_dir=log_dir)
 

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,6)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(Dropout(0.12))
# model.add(LSTM(256,return_sequences=True, activation='relu'))  #added layer
# model.add(LSTM(128, return_sequences=True, activation='relu')) #added layer
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(out.shape[0], activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=1000, shuffle = True)